In [1]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
#from google.colab import drive

#drive.mount('/content/drive')

in_data = loadmat('face_emotion_data.mat')

X = in_data['X']
y = in_data['y']

In [2]:
#print(X.shape)

best_r = -1
best_error = 10**32

for r in range(1,10):
  error_rate_sum = 0.0
  for i in range(8):
      for j in range(8):
          if i == j: continue
          test_idx_1 = np.arange(i*16, (i+1)*16)
          test_idx_2 = np.arange(j*16, (j+1)*16)
          train_idx = np.setdiff1d(np.arange(128), test_idx_1)
          train_idx = np.setdiff1d(train_idx, test_idx_2)
          X_train, y_train = X[train_idx, :], y[train_idx, :]
          X_test_1, y_test_1 = X[test_idx_1, :], y[test_idx_1, :]
          X_test_2, y_test_2 = X[test_idx_2, :], y[test_idx_2, :]

          U, s, VT = np.linalg.svd(X_train, full_matrices=False)
          s_inv = np.vectorize(lambda s: 1/s)(s)
          #w = VT.T @ np.diag(s_inv) @ U.T @ y_train
          w = VT.T[:,:r] @ np.diag(s_inv)[:r,:r] @ U.T[:r] @ y_train
          y_pred = X_test_1 @ w
          y_pred_class = np.vectorize(lambda x : -1 if x <= 0 else 1)(y_pred)

          error = np.count_nonzero(y_pred_class != y_test_1)
          error_rate = error/16.0
          error_rate_sum += error_rate

  error_rate_avg = error_rate_sum / 56.0
  if error_rate_avg < best_error:
    best_error = error_rate_avg
    best_r = r

#Test on holdout, for best r

error_rate_sum = 0.0
for i in range(8):
    for j in range(8):
        if i == j: continue
        test_idx_1 = np.arange(i*16, (i+1)*16)
        test_idx_2 = np.arange(j*16, (j+1)*16)
        train_idx = np.setdiff1d(np.arange(128), test_idx_1)
        train_idx = np.setdiff1d(train_idx, test_idx_2)
        X_train, y_train = X[train_idx, :], y[train_idx, :]
        X_test_1, y_test_1 = X[test_idx_1, :], y[test_idx_1, :]
        X_test_2, y_test_2 = X[test_idx_2, :], y[test_idx_2, :]

        U, s, VT = np.linalg.svd(X_train, full_matrices=False)
        s_inv = np.vectorize(lambda s: 1/s)(s)
        #w = VT.T @ np.diag(s_inv) @ U.T @ y_train
        w = VT.T[:,:best_r] @ np.diag(s_inv)[:best_r,:best_r] @ U.T[:best_r] @ y_train
        y_pred = X_test_2 @ w
        y_pred_class = np.vectorize(lambda x : -1 if x <= 0 else 1)(y_pred)

        error = np.count_nonzero(y_pred_class != y_test_2)
        error_rate = error/16.0
        error_rate_sum += error_rate

error_rate_avg = error_rate_sum / 56.0
print(best_r)
print(error_rate_avg)

6
0.04352678571428571


In [6]:
#Part b

best_lam = -50
best_error = 10**32

for r in range(-2,5):
  error_rate_sum = 0.0
  for i in range(8):
      for j in range(8):
          if i == j: continue
          test_idx_1 = np.arange(i*16, (i+1)*16)
          test_idx_2 = np.arange(j*16, (j+1)*16)
          train_idx = np.setdiff1d(np.arange(128), test_idx_1)
          train_idx = np.setdiff1d(train_idx, test_idx_2)
          X_train, y_train = X[train_idx, :], y[train_idx, :]
          X_test_1, y_test_1 = X[test_idx_1, :], y[test_idx_1, :]
          X_test_2, y_test_2 = X[test_idx_2, :], y[test_idx_2, :]

          U, s, VT = np.linalg.svd(X_train, full_matrices=False)
          s_squared = np.vectorize(lambda s: s*s)(s)
          if r != -2:
            lam_I = np.diag([2 ** r for _ in range(len(s))])
          else:
            lam_I = np.diag([0 for _ in range(len(s))])
          w = VT.T @ (np.linalg.inv(np.diag(s_squared) + lam_I)) @ np.diag(s) @ U.T @ y_train

          y_pred = X_test_1 @ w
          y_pred_class = np.vectorize(lambda x : -1 if x <= 0 else 1)(y_pred)

          error = np.count_nonzero(y_pred_class != y_test_1)
          error_rate = error/16.0
          error_rate_sum += error_rate

  error_rate_avg = error_rate_sum / 56.0
  print(error_rate_avg)
  if error_rate_avg < best_error:
    best_error = error_rate_avg
    best_lam = r

#Test on holdout, for best r

error_rate_sum = 0.0
for i in range(8):
      for j in range(8):
          if i == j: continue
          test_idx_1 = np.arange(i*16, (i+1)*16)
          test_idx_2 = np.arange(j*16, (j+1)*16)
          train_idx = np.setdiff1d(np.arange(128), test_idx_1)
          train_idx = np.setdiff1d(train_idx, test_idx_2)
          X_train, y_train = X[train_idx, :], y[train_idx, :]
          X_test_1, y_test_1 = X[test_idx_1, :], y[test_idx_1, :]
          X_test_2, y_test_2 = X[test_idx_2, :], y[test_idx_2, :]

          U, s, VT = np.linalg.svd(X_train, full_matrices=False)
          s_squared = np.vectorize(lambda s: s*s)(s)
          if r != -2:
            lam_I = np.diag([2 ** r for _ in range(len(s))])
          else:
            lam_I = np.diag([0 for _ in range(len(s))])
          w = VT.T @ (np.linalg.inv(np.diag(s_squared) + lam_I)) @ np.diag(s) @ U.T @ y_train

          y_pred = X_test_1 @ w
          y_pred_class = np.vectorize(lambda x : -1 if x <= 0 else 1)(y_pred)

          error = np.count_nonzero(y_pred_class != y_test_1)
          error_rate = error/16.0
          error_rate_sum += error_rate

error_rate_avg = error_rate_sum / 56.0
print()
print(best_r)
print(error_rate_avg)

0.04575892857142857
0.04352678571428571
0.04352678571428571
0.04575892857142857
0.049107142857142856
0.05133928571428571
0.05357142857142857

-1
0.05357142857142857
